# Credit Risk Prediction Analysis

**Author**: Rafsamjani Anugrah  
**Date**: 2024  
**Project**: Rakamin VIX Program - ID/X Partners Credit Risk Prediction

## Project Overview

This notebook presents a comprehensive analysis of loan data from Lending Club (2007-2014) to build a credit risk prediction model. The analysis includes:

1. **Data Exploration and Preprocessing**
2. **Exploratory Data Analysis (EDA)**
3. **Feature Engineering**
4. **Model Development and Training**
5. **Model Evaluation and Selection**
6. **Business Insights and Recommendations**

### Business Context

ID/X Partners is a lending company that needs to assess creditworthiness of loan applicants to minimize financial losses while maximizing profitable lending opportunities.

## 1. Setup and Data Loading

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import shap
import joblib

# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Libraries imported successfully!")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Load the dataset
try:
    # Try loading from the data directory
    df = pd.read_csv('../data/raw/loan_data_2007_2014.csv')
    print(f"Dataset loaded successfully!")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
except FileNotFoundError:
    print("Dataset file not found. Please ensure the data is in the correct location.")
    print("Expected path: ../data/raw/loan_data_2007_2014.csv")
    # For demonstration, create a sample dataset structure
    print("\nCreating sample dataset structure for demonstration...")
    # This would be replaced with actual data loading

# Display basic information
if 'df' in locals():
    print("\n" + "="*50)
    print("DATASET INFORMATION")
    print("="*50)
    print(f"Total Records: {df.shape[0]:,}")
    print(f"Total Features: {df.shape[1]}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

In [ ]:
# Display first few rows and column information
if 'df' in locals():
    print("First 5 rows:")
    display(df.head())
    
    print("\nColumn Information:")
    print(df.info())

## 2. Data Understanding and Quality Assessment

In [ ]:
# Analyze data types and missing values
if 'df' in locals():
    print("""=""*50)
    print("DATA QUALITY ASSESSMENT")
    print("""=""*50)
    
    # Missing value analysis
    missing_analysis = pd.DataFrame({
        'Column': df.columns,
        'Data Type': df.dtypes.values,
        'Missing Count': df.isnull().sum().values,
        'Missing %': (df.isnull().sum() / len(df) * 100).round(2)
    })
    
    # Sort by missing percentage
    missing_analysis = missing_analysis.sort_values('Missing %', ascending=False)
    
    # Display columns with missing values
    missing_cols = missing_analysis[missing_analysis['Missing Count'] > 0]
    
    print(f"\nColumns with missing values: {len(missing_cols)} out of {len(df.columns)}")
    print("\nTop 20 columns with missing values:")
    display(missing_cols.head(20))

In [ ]:
# Analyze target variable distribution
if 'df' in locals():
    print("""=""*50)
    print("TARGET VARIABLE ANALYSIS")
    print("""=""*50)
    
    # Loan status distribution
    if 'loan_status' in df.columns:
        status_counts = df['loan_status'].value_counts()
        status_percentages = (status_counts / len(df) * 100).round(2)
        
        print("\nLoan Status Distribution:")
        for status, count in status_counts.items():
            percentage = status_percentages[status]
            print(f"  {status}: {count:,} ({percentage}%)")
        
        # Visualize loan status distribution
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Bar plot
        status_counts.plot(kind='bar', ax=ax1)
        ax1.set_title('Loan Status Distribution')
        ax1.set_xlabel('Loan Status')
        ax1.set_ylabel('Count')
        ax1.tick_params(axis='x', rotation=45)
        
        # Pie chart
        ax2.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%')
        ax2.set_title('Loan Status Proportion')
        
        plt.tight_layout()
        plt.show()
        
        # Focus on completed loans (Fully Paid and Charged Off)
        completed_loans = ['Fully Paid', 'Charged Off']
        df_completed = df[df['loan_status'].isin(completed_loans)].copy()
        
        print(f"\nCompleted loans (Fully Paid + Charged Off): {len(df_completed):,} ({len(df_completed)/len(df)*100:.1f}%)")
        
        # Default rate among completed loans
        if len(df_completed) > 0:
            default_rate = (df_completed['loan_status'] == 'Charged Off').mean() * 100
            print(f"Default rate among completed loans: {default_rate:.1f}%")
    else:
        print("Loan status column not found in dataset.")

## 3. Data Preprocessing

In [ ]:
# Filter for completed loans and prepare target variable
if 'df' in locals():
    print("""=""*50)
    print("DATA PREPROCESSING")
    print("""=""*50)
    
    # Filter for completed loans only
    completed_loans = ['Fully Paid', 'Charged Off']
    df_clean = df[df['loan_status'].isin(completed_loans)].copy()
    
    print(f"Original dataset: {len(df):,} records")
    print(f"Filtered dataset: {len(df_clean):,} records")
    print(f"Records removed: {len(df) - len(df_clean):,} ({(len(df) - len(df_clean))/len(df)*100:.1f}%)")
    
    # Convert target variable to binary
    df_clean['loan_status_binary'] = (df_clean['loan_status'] == 'Charged Off').astype(int)
    
    print(f"\nTarget variable distribution:")
    target_counts = df_clean['loan_status_binary'].value_counts()
    for status, count in target_counts.items():
        status_name = 'Charged Off' if status == 1 else 'Fully Paid'
        percentage = count / len(df_clean) * 100
        print(f"  {status_name}: {count:,} ({percentage:.1f}%)")
    
    # Class imbalance ratio
    imbalance_ratio = target_counts[0] / target_counts[1]
    print(f"\nClass imbalance ratio (Non-default:Default): {imbalance_ratio:.1f}:1")

In [ ]:
# Clean and transform key variables
if 'df_clean' in locals():
    print("Cleaning and transforming variables...")
    
    # Convert percentage columns
    percentage_cols = ['int_rate', 'revol_util']
    for col in percentage_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].astype(str).str.replace('%', '').astype(float)
            print(f"Converted {col} to numeric")
    
    # Clean employment length
    if 'emp_length' in df_clean.columns:
        emp_length_mapping = {
            '< 1 year': 0.5, '1 year': 1, '2 years': 2, '3 years': 3,
            '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7,
            '8 years': 8, '9 years': 9, '10+ years': 10
        }
        df_clean['emp_length_numeric'] = df_clean['emp_length'].map(emp_length_mapping)
        print(f"Converted employment length to numeric")
    
    # Convert date columns
    date_cols = ['issue_d', 'earliest_cr_line']
    for col in date_cols:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], format='%b-%y', errors='coerce')
            print(f"Converted {col} to datetime")
    
    # Create derived features
    if 'fico_range_low' in df_clean.columns and 'fico_range_high' in df_clean.columns:
        df_clean['fico_avg'] = (df_clean['fico_range_low'] + df_clean['fico_range_high']) / 2
        print("Created FICO score average")
    
    # Financial ratios
    if 'annual_inc' in df_clean.columns and 'loan_amnt' in df_clean.columns:
        df_clean['loan_to_income_ratio'] = df_clean['loan_amnt'] / (df_clean['annual_inc'] / 12)
        print("Created loan-to-income ratio")
    
    # Credit age
    if 'earliest_cr_line' in df_clean.columns and 'issue_d' in df_clean.columns:
        df_clean['credit_age_years'] = (
            df_clean['issue_d'] - df_clean['earliest_cr_line']
        ).dt.days / 365.25
        print("Created credit age in years")
    
    print(f"\nData cleaning completed. Final shape: {df_clean.shape}")

## 4. Exploratory Data Analysis

### 4.1 Loan Characteristics Analysis

In [ ]:
# Analyze loan characteristics by default status
if 'df_clean' in locals():
    print("""=""*50)
    print("LOAN CHARACTERISTICS ANALYSIS")
    print("""=""*50)
    
    # Key loan characteristics
    loan_chars = ['loan_amnt', 'int_rate', 'term', 'purpose', 'grade']
    available_chars = [col for col in loan_chars if col in df_clean.columns]
    
    for char in available_chars:
        print(f"\n--- {char.upper()} ANALYSIS ---")
        
        if df_clean[char].dtype in ['object', 'category']:
            # Categorical analysis
            cross_tab = pd.crosstab(df_clean[char], df_clean['loan_status_binary'], normalize='index') * 100
            cross_tab.columns = ['Fully Paid Rate', 'Default Rate']
            
            print(f"Default rates by {char}:")
            display(cross_tab.sort_values('Default Rate', ascending=False))
            
            # Visualization
            if len(df_clean[char].unique()) <= 10:  # Only plot for reasonable number of categories
                plt.figure(figsize=(12, 6))
                cross_tab['Default Rate'].sort_values().plot(kind='bar')
                plt.title(f'Default Rate by {char}')
                plt.xlabel(char)
                plt.ylabel('Default Rate (%)')
                plt.xticks(rotation=45)
                plt.show()
        else:
            # Numerical analysis
            stats_by_status = df_clean.groupby('loan_status_binary')[char].describe()
            stats_by_status.index = ['Fully Paid', 'Charged Off']
            
            print(f"{char} statistics by loan status:")
            display(stats_by_status)
            
            # Visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Box plot
            df_clean.boxplot(column=char, by='loan_status_binary', ax=ax1)
            ax1.set_title(f'{char} by Loan Status')
            ax1.set_xlabel('Loan Status (0=Fully Paid, 1=Charged Off)')
            
            # Distribution
            for status in [0, 1]:
                status_name = 'Fully Paid' if status == 0 else 'Charged Off'
                df_clean[df_clean['loan_status_binary'] == status][char].hist(
                    alpha=0.7, label=status_name, ax=ax2, bins=30
                )
            ax2.set_title(f'Distribution of {char}')
            ax2.legend()
            
            plt.tight_layout()
            plt.show()

### 4.2 Borrower Demographics Analysis

In [ ]:
# Analyze borrower demographics
if 'df_clean' in locals():
    print("""=""*50)
    print("BORROWER DEMOGRAPHICS ANALYSIS")
    print("""=""*50)
    
    # Key borrower characteristics
    borrower_chars = ['annual_inc', 'emp_length_numeric', 'home_ownership', 'dti', 'fico_avg']
    available_borrower_chars = [col for col in borrower_chars if col in df_clean.columns]
    
    for char in available_borrower_chars:
        print(f"\n--- {char.upper()} ANALYSIS ---")
        
        if df_clean[char].dtype in ['object', 'category']:
            # Categorical analysis
            cross_tab = pd.crosstab(df_clean[char], df_clean['loan_status_binary'], normalize='index') * 100
            cross_tab.columns = ['Fully Paid Rate', 'Default Rate']
            
            print(f"Default rates by {char}:")
            display(cross_tab.sort_values('Default Rate', ascending=False))
            
            # Visualization
            plt.figure(figsize=(10, 6))
            cross_tab['Default Rate'].sort_values().plot(kind='bar', color='coral')
            plt.title(f'Default Rate by {char}')
            plt.xlabel(char)
            plt.ylabel('Default Rate (%)')
            plt.xticks(rotation=45)
            plt.show()
        else:
            # Numerical analysis
            stats_by_status = df_clean.groupby('loan_status_binary')[char].describe()
            stats_by_status.index = ['Fully Paid', 'Charged Off']
            
            print(f"{char} statistics by loan status:")
            display(stats_by_status)
            
            # Calculate correlation with default
            correlation = df_clean[char].corr(df_clean['loan_status_binary'])
            print(f"Correlation with default: {correlation:.3f}")
            
            # Visualization
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            
            # Box plot
            df_clean.boxplot(column=char, by='loan_status_binary', ax=axes[0,0])
            axes[0,0].set_title(f'{char} by Loan Status')
            
            # Histogram
            axes[0,1].hist(df_clean[df_clean['loan_status_binary'] == 0][char], 
                          alpha=0.7, label='Fully Paid', bins=30, color='green')
            axes[0,1].hist(df_clean[df_clean['loan_status_binary'] == 1][char], 
                          alpha=0.7, label='Charged Off', bins=30, color='red')
            axes[0,1].set_title(f'Distribution of {char}')
            axes[0,1].legend()
            
            # Density plot
            df_clean[df_clean['loan_status_binary'] == 0][char].plot.density(
                ax=axes[1,0], label='Fully Paid', color='green')
            df_clean[df_clean['loan_status_binary'] == 1][char].plot.density(
                ax=axes[1,0], label='Charged Off', color='red')
            axes[1,0].set_title(f'Density Plot of {char}')
            axes[1,0].legend()
            
            # Default rate by quantiles
            try:
                quantiles = pd.qcut(df_clean[char], q=10, duplicates='drop')
                default_by_quantile = df_clean.groupby(quantiles)['loan_status_binary'].mean() * 100
                default_by_quantile.plot(kind='bar', ax=axes[1,1], color='purple')
                axes[1,1].set_title(f'Default Rate by {char} Quantiles')
                axes[1,1].set_xlabel(f'{char} Quantiles')
                axes[1,1].set_ylabel('Default Rate (%)')
                axes[1,1].tick_params(axis='x', rotation=45)
            except:
                axes[1,1].text(0.5, 0.5, 'Cannot create quantiles', 
                              ha='center', va='center', transform=axes[1,1].transAxes)
            
            plt.tight_layout()
            plt.show()

### 4.3 Credit History Analysis

In [ ]:
# Analyze credit history variables
if 'df_clean' in locals():
    print("""=""*50)
    print("CREDIT HISTORY ANALYSIS")
    print("""=""*50)
    
    # Credit history variables
    credit_vars = ['delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 
                  'revol_bal', 'revol_util', 'credit_age_years']
    available_credit_vars = [col for col in credit_vars if col in df_clean.columns]
    
    # Create credit score categories
    if 'fico_avg' in df_clean.columns:
        df_clean['fico_category'] = pd.cut(
            df_clean['fico_avg'],
            bins=[0, 680, 720, 760, 850],
            labels=['Fair (680-)', 'Good (680-720)', 'Very Good (720-760)', 'Excellent (760+)']
        )
        
        # Default rate by FICO category
        fico_default_rates = df_clean.groupby('fico_category')['loan_status_binary'].mean() * 100
        print("Default rate by FICO category:")
        display(fico_default_rates)
        
        # Visualization
        plt.figure(figsize=(10, 6))
        fico_default_rates.sort_values().plot(kind='bar', color='navy')
        plt.title('Default Rate by FICO Score Category')
        plt.ylabel('Default Rate (%)')
        plt.xticks(rotation=45)
        plt.show()
    
    # Analyze other credit variables
    for var in available_credit_vars:
        if var == 'fico_avg' or var == 'fico_category':
            continue
            
        print(f"\n--- {var.upper()} ANALYSIS ---")
        
        # Basic statistics
        print(f"Statistics for {var}:")
        print(df_clean[var].describe())
        
        # Default rate by categories
        if df_clean[var].nunique() <= 10:  # Discrete variable
            default_by_var = df_clean.groupby(var)['loan_status_binary'].mean() * 100
            print(f"\nDefault rate by {var}:")
            display(default_by_var)
            
            # Visualization
            plt.figure(figsize=(10, 6))
            default_by_var.plot(kind='bar', color='darkorange')
            plt.title(f'Default Rate by {var}')
            plt.ylabel('Default Rate (%)')
            plt.show()
        else:  # Continuous variable
            # Create bins
            try:
                df_clean[f'{var}_binned'] = pd.qcut(df_clean[var], q=5, duplicates='drop')
                default_by_bins = df_clean.groupby(f'{var}_binned')['loan_status_binary'].mean() * 100
                
                print(f"\nDefault rate by {var} quintiles:")
                display(default_by_bins)
                
                # Visualization
                plt.figure(figsize=(10, 6))
                default_by_bins.plot(kind='bar', color='darkorange')
                plt.title(f'Default Rate by {var} Quintiles')
                plt.ylabel('Default Rate (%)')
                plt.xticks(rotation=45)
                plt.show()
            except Exception as e:
                print(f"Could not create bins for {var}: {str(e)}")

### 4.4 Correlation Analysis

In [ ]:
# Correlation analysis
if 'df_clean' in locals():
    print("""=""*50)
    print("CORRELATION ANALYSIS")
    print("""=""*50)
    
    # Select numerical columns for correlation
    numerical_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    # Remove some ID or highly specific columns
    numerical_cols = [col for col in numerical_cols if 
                     not any(skip in col.lower() for skip in ['id', 'member_id', 'policy_code'])]
    
    # Calculate correlation matrix
    correlation_matrix = df_clean[numerical_cols].corr()
    
    # Focus on correlation with target variable
    target_correlation = correlation_matrix['loan_status_binary'].sort_values(ascending=False)
    
    print("\nCorrelation with Default (loan_status_binary):")
    print(target_correlation)
    
    # Top correlations with target
    top_correlations = target_correlation.abs().sort_values(ascending=False).head(15)
    print(f"\nTop 15 variables most correlated with default:")
    display(top_correlations)
    
    # Heatmap of top correlated features
    top_features = top_correlations.index.tolist()
    
    plt.figure(figsize=(12, 10))
    correlation_subset = df_clean[top_features].corr()
    
    # Create heatmap
    mask = np.triu(np.ones_like(correlation_subset, dtype=bool))
    sns.heatmap(correlation_subset, 
                annot=True, 
                cmap='coolwarm', 
                center=0,
                mask=mask,
                fmt='.2f',
                square=True)
    plt.title('Correlation Heatmap of Top Features')
    plt.tight_layout()
    plt.show()
    
    # Identify multicollinearity (high correlations between features)
    print("\nHigh correlations between features (|r| > 0.7):")
    high_corr_pairs = []
    
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr_value = correlation_matrix.iloc[i, j]
            if abs(corr_value) > 0.7:
                var1 = correlation_matrix.columns[i]
                var2 = correlation_matrix.columns[j]
                high_corr_pairs.append((var1, var2, corr_value))
    
    if high_corr_pairs:
        for var1, var2, corr in high_corr_pairs:
            print(f"  {var1} ↔ {var2}: {corr:.3f}")
    else:
        print("  No high correlations found between features.")

## 5. Feature Engineering and Selection

In [ ]:
# Feature engineering
if 'df_clean' in locals():
    print("""=""*50)
    print("FEATURE ENGINEERING")
    print("""=""*50)
    
    df_features = df_clean.copy()
    
    # Create additional derived features
    print("Creating additional engineered features...")
    
    # Effective DTI (including new loan payment)
    if all(col in df_features.columns for col in ['dti', 'installment', 'annual_inc']):
        monthly_income = df_features['annual_inc'] / 12
        new_payment_dti = (df_features['installment'] / monthly_income) * 100
        df_features['effective_dti'] = df_features['dti'] + new_payment_dti
        print("Created effective DTI (including new loan payment)")
    
    # Monthly debt burden
    if all(col in df_features.columns for col in ['installment', 'annual_inc']):
        df_features['monthly_debt_burden'] = df_features['installment'] / (df_features['annual_inc'] / 12)
        print("Created monthly debt burden ratio")
    
    # Employment stability score
    if all(col in df_features.columns for col in ['emp_length_numeric', 'home_ownership']):
        df_features['employment_stability'] = (
            df_features['emp_length_numeric'] * 7 +
            (df_features['home_ownership'].isin(['MORTGAGE', 'OWN']) * 20)
        )
        print("Created employment stability score")
    
    # Credit utilization ratio (if available)
    if 'revol_util' in df_features.columns:
        df_features['high_utilization'] = (df_features['revol_util'] > 80).astype(int)
        print("Created high utilization flag")
    
    # Risk flags
    if 'fico_avg' in df_features.columns:
        df_features['low_fico'] = (df_features['fico_avg'] < 680).astype(int)
        print("Created low FICO flag")
    
    if 'dti' in df_features.columns:
        df_features['high_dti'] = (df_features['dti'] > 30).astype(int)
        print("Created high DTI flag")
    
    print(f"\nFeature engineering completed. Final shape: {df_features.shape}")
    
    # Display new features
    new_features = [col for col in df_features.columns if col not in df_clean.columns]
    print(f"\nCreated {len(new_features)} new features:")
    for feature in new_features:
        print(f"  - {feature}")

In [ ]:
# Feature selection
if 'df_features' in locals():
    print("""=""*50)
    print("FEATURE SELECTION")
    print("""=""*50)
    
    # Select key features for modeling
    # Based on EDA, domain knowledge, and correlation analysis
    
    key_features = [
        # Loan characteristics
        'loan_amnt', 'term', 'int_rate', 'installment', 'grade', 'purpose',
        
        # Borrower information
        'annual_inc', 'emp_length_numeric', 'home_ownership', 'verification_status', 'dti',
        
        # Credit history
        'fico_avg', 'credit_age_years', 'delinq_2yrs', 'inq_last_6mths', 
        'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
        
        # Engineered features
        'loan_to_income_ratio', 'effective_dti', 'monthly_debt_burden',
        'employment_stability'
    ]
    
    # Filter available features
    available_features = [f for f in key_features if f in df_features.columns]
    
    print(f"Selected {len(available_features)} features for modeling:")
    for i, feature in enumerate(available_features, 1):
        print(f"  {i:2d}. {feature}")
    
    # Create final feature matrix
    X = df_features[available_features].copy()
    y = df_features['loan_status_binary']
    
    print(f"\nFeature matrix shape: {X.shape}")
    print(f"Target vector shape: {y.shape}")
    
    # Check for missing values in selected features
    missing_in_features = X.isnull().sum()
    if missing_in_features.sum() > 0:
        print(f"\nMissing values in selected features:")
        display(missing_in_features[missing_in_features > 0])
    else:
        print("\nNo missing values in selected features.")

## 6. Model Development

In [ ]:
# Prepare data for modeling
if 'X' in locals() and 'y' in locals():
    print("""=""*50)
    print("DATA PREPARATION FOR MODELING")
    print("""=""*50)
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )
    
    print(f"Training set: {X_train.shape[0]:,} samples")
    print(f"Test set: {X_test.shape[0]:,} samples")
    print(f"Training set default rate: {y_train.mean()*100:.1f}%")
    print(f"Test set default rate: {y_test.mean()*100:.1f}%")
    
    # Preprocess categorical variables
    def preprocess_features(X_train, X_test):
        """Preprocess features for modeling"""
        X_train_processed = X_train.copy()
        X_test_processed = X_test.copy()
        
        # Handle categorical variables
        categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
        numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
        
        # Encode categorical variables
        for col in categorical_cols:
            if col == 'grade':
                # Ordinal encoding for loan grades
                grade_mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
                X_train_processed[col] = X_train[col].map(grade_mapping).fillna(4)
                X_test_processed[col] = X_test[col].map(grade_mapping).fillna(4)
            else:
                # One-hot encoding for other categorical variables
                encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
                
                # Fit on training data
                train_encoded = encoder.fit_transform(X_train[[col]])
                feature_names = [f"{col}_{cat}" for cat in encoder.categories_[0]]
                
                train_encoded_df = pd.DataFrame(train_encoded, columns=feature_names, index=X_train.index)
                X_train_processed = pd.concat([X_train_processed.drop(col, axis=1), train_encoded_df], axis=1)
                
                # Transform test data
                test_encoded = encoder.transform(X_test[[col]])
                test_encoded_df = pd.DataFrame(test_encoded, columns=feature_names, index=X_test.index)
                X_test_processed = pd.concat([X_test_processed.drop(col, axis=1), test_encoded_df], axis=1)
        
        # Handle missing values in numerical columns
        for col in numerical_cols:
            if X_train_processed[col].isnull().any():
                median_value = X_train_processed[col].median()
                X_train_processed[col].fillna(median_value, inplace=True)
                X_test_processed[col].fillna(median_value, inplace=True)
        
        # Scale numerical features
        scaler = StandardScaler()
        numerical_cols_processed = X_train_processed.select_dtypes(include=['int64', 'float64']).columns
        
        X_train_processed[numerical_cols_processed] = scaler.fit_transform(X_train_processed[numerical_cols_processed])
        X_test_processed[numerical_cols_processed] = scaler.transform(X_test_processed[numerical_cols_processed])
        
        return X_train_processed, X_test_processed, scaler, encoder
    
    X_train_processed, X_test_processed, scaler, encoders = preprocess_features(X_train, X_test)
    
    print(f"\nProcessed training set shape: {X_train_processed.shape}")
    print(f"Processed test set shape: {X_test_processed.shape}")
    
    # Handle class imbalance with SMOTE
    print("\nApplying SMOTE to handle class imbalance...")
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)
    
    print(f"Original training set: {X_train_processed.shape}")
    print(f"Resampled training set: {X_train_resampled.shape}")
    print(f"Original target distribution: {pd.Series(y_train).value_counts().to_dict()}")
    print(f"Resampled target distribution: {pd.Series(y_train_resampled).value_counts().to_dict()}")

In [ ]:
# Train multiple models
if 'X_train_resampled' in locals():
    print("""=""*50)
    print("MODEL TRAINING")
    print("""=""*50)
    
    # Define models
    models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Random Forest': RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            random_state=42
        ),
        'XGBoost': xgb.XGBClassifier(
            n_estimators=200,
            max_depth=10,
            learning_rate=0.1,
            random_state=42
        )
    }
    
    # Train and evaluate models
    results = {}
    best_model = None
    best_score = 0
    
    for name, model in models.items():
        print(f"\n--- Training {name} ---")
        
        # Train model
        model.fit(X_train_resampled, y_train_resampled)
        
        # Make predictions
        y_pred = model.predict(X_test_processed)
        y_proba = model.predict_proba(X_test_processed)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)
        
        metrics = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'roc_auc': roc_auc
        }
        
        results[name] = {
            'model': model,
            'metrics': metrics,
            'predictions': y_pred,
            'probabilities': y_proba
        }
        
        print(f"Accuracy:  {accuracy:.3f}")
        print(f"Precision: {precision:.3f}")
        print(f"Recall:    {recall:.3f}")
        print(f"F1-Score:  {f1:.3f}")
        print(f"ROC-AUC:   {roc_auc:.3f}")
        
        # Track best model
        if roc_auc > best_score:
            best_score = roc_auc
            best_model = name
            best_model_instance = model
    
    print(f"\n" + "="*50)
    print(f"BEST MODEL: {best_model} with ROC-AUC: {best_score:.3f}")
    print("="*50)

## 7. Model Evaluation and Analysis

In [ ]:
# Detailed evaluation of the best model
if 'results' in locals() and 'best_model' in locals():
    print("""=""*50)
    print(f"DETAILED EVALUATION - {best_model.upper()}")
    print("""=""*50)
    
    best_result = results[best_model]
    model = best_result['model']
    y_pred = best_result['predictions']
    y_proba = best_result['probabilities']
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print("\nConfusion Matrix:")
    print(f"True Negatives:  {tn:6d} (Good loans correctly approved)")
    print(f"False Positives: {fp:6d} (Good loans incorrectly rejected)")
    print(f"False Negatives: {fn:6d} (Bad loans incorrectly approved)")
    print(f"True Positives:  {tp:6d} (Bad loans correctly rejected)")
    
    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Fully Paid', 'Charged Off'],
                yticklabels=['Fully Paid', 'Charged Off'])
    plt.title(f'Confusion Matrix - {best_model}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # ROC Curve
    fpr, tpr, thresholds = roc_curve(y_test, y_proba)
    
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC Curve (AUC = {best_result["metrics"]["roc_auc"]:.3f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {best_model}')
    plt.legend(loc="lower right")
    plt.show()
    
    # Feature Importance (for tree-based models)
    if hasattr(model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'feature': X_test_processed.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nTop 15 Most Important Features:")
        display(feature_importance.head(15))
        
        # Visualize feature importance
        plt.figure(figsize=(12, 8))
        top_features = feature_importance.head(15)
        plt.barh(range(len(top_features)), top_features['importance'])
        plt.yticks(range(len(top_features)), top_features['feature'])
        plt.xlabel('Feature Importance')
        plt.title(f'Top 15 Feature Importance - {best_model}')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
    
    # Classification Report
    print("\nDetailed Classification Report:")
    print(classification_report(y_test, y_pred, 
                              target_names=['Fully Paid', 'Charged Off']))

In [ ]:
# Model comparison visualization
if 'results' in locals():
    print("""=""*50)
    print("MODEL COMPARISON")
    print("""=""*50)
    
    # Create comparison DataFrame
    comparison_data = []
    for name, result in results.items():
        metrics = result['metrics']
        comparison_data.append({
            'Model': name,
            'Accuracy': metrics['accuracy'],
            'Precision': metrics['precision'],
            'Recall': metrics['recall'],
            'F1-Score': metrics['f1'],
            'ROC-AUC': metrics['roc_auc']
        })
    
    comparison_df = pd.DataFrame(comparison_data).set_index('Model')
    
    print("Model Performance Comparison:")
    display(comparison_df)
    
    # Visualize comparison
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    metrics_list = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
    
    for i, metric in enumerate(metrics_list):
        row = i // 3
        col = i % 3
        
        comparison_df[metric].sort_values().plot(kind='bar', ax=axes[row, col], color='skyblue')
        axes[row, col].set_title(f'{metric} Comparison')
        axes[row, col].set_ylabel(metric)
        axes[row, col].tick_params(axis='x', rotation=45)
    
    # Hide empty subplot
    axes[1, 2].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Create radar chart for overall comparison
    from math import pi
    
    # Number of variables
    categories = metrics_list
    N = len(categories)
    
    # Angle for each axis
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Initialize radar chart
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Plot each model
    colors = ['red', 'blue', 'green']
    for i, (model_name, row) in enumerate(comparison_df.iterrows()):
        values = row[categories].values.flatten().tolist()
        values += values[:1]  # Complete the circle
        
        ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[i])
        ax.fill(angles, values, alpha=0.25, color=colors[i])
    
    # Add labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_ylim(0, 1)
    ax.set_title('Model Performance Radar Chart', size=16, pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    plt.show()

## 8. Business Impact Analysis

In [ ]:
# Business impact simulation
if 'results' in locals() and 'best_model' in locals():
    print("""=""*50)
    print("BUSINESS IMPACT ANALYSIS")
    print("""=""*50)
    
    best_result = results[best_model]
    y_pred = best_result['predictions']
    y_proba = best_result['probabilities']
    
    # Business assumptions
    avg_loan_amount = X_test['loan_amnt'].mean() if 'loan_amnt' in X_test.columns else 15000
    avg_interest_rate = X_test['int_rate'].mean() if 'int_rate' in X_test.columns else 0.13
    default_loss_rate = 0.60  # 60% loss on default
    cost_of_capital = 0.05   # 5% cost of funds
    
    # Confusion matrix values
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    # Calculate financial impact
    # Benefits
    profit_from_good_loans = tn * avg_loan_amount * avg_interest_rate
    savings_from_avoided_defaults = tp * avg_loan_amount * default_loss_rate
    
    # Costs
    opportunity_cost = fp * avg_loan_amount * (avg_interest_rate - cost_of_capital)
    default_losses = fn * avg_loan_amount * default_loss_rate
    
    # Net impact
    total_benefit = profit_from_good_loans + savings_from_avoided_defaults
    total_cost = opportunity_cost + default_losses
    net_financial_impact = total_benefit - total_cost
    
    print("\nBusiness Impact Simulation:")
    print(f"Average loan amount: ${avg_loan_amount:,.0f}")
    print(f"Average interest rate: {avg_interest_rate:.1%}")
    print(f"Default loss rate: {default_loss_rate:.1%}")
    print(f"Cost of capital: {cost_of_capital:.1%}")
    
    print("\n" + "="*50)
    print("FINANCIAL IMPACT BREAKDOWN")
    print("="*50)
    
    print("\n💰 BENEFITS:")
    print(f"  Profit from good loans:        ${profit_from_good_loans:,.0f}")
    print(f"  Savings from avoided defaults:  ${savings_from_avoided_defaults:,.0f}")
    print(f"  Total Benefits:                 ${total_benefit:,.0f}")
    
    print("\n💸 COSTS:")
    print(f"  Opportunity cost (false pos):  ${opportunity_cost:,.0f}")
    print(f"  Default losses (false neg):    ${default_losses:,.0f}")
    print(f"  Total Costs:                    ${total_cost:,.0f}")
    
    print("\n📊 NET FINANCIAL IMPACT:")
    print(f"  Net Impact:                     ${net_financial_impact:,.0f}")
    
    if net_financial_impact > 0:
        print(f"  ✅ Model creates ${net_financial_impact:,.0f} positive financial impact")
    else:
        print(f"  ❌ Model results in ${abs(net_financial_impact):,.0f} negative financial impact")
    
    # Calculate ROI
    roi = (net_financial_impact / total_cost) * 100 if total_cost > 0 else 0
    print(f"  Return on Investment: {roi:.1f}%")
    
    # Optimal threshold analysis
    print("\n" + "="*50)
    print("OPTIMAL THRESHOLD ANALYSIS")
    print("="*50)
    
    thresholds = np.arange(0.1, 0.9, 0.1)
    threshold_analysis = []
    
    for threshold in thresholds:
        y_pred_thresh = (y_proba >= threshold).astype(int)
        tn_t, fp_t, fn_t, tp_t = confusion_matrix(y_test, y_pred_thresh).ravel()
        
        # Calculate financial impact for this threshold
        benefits = (tn_t * avg_loan_amount * avg_interest_rate + 
                   tp_t * avg_loan_amount * default_loss_rate)
        costs = (fp_t * avg_loan_amount * (avg_interest_rate - cost_of_capital) + 
                fn_t * avg_loan_amount * default_loss_rate)
        net_impact = benefits - costs
        
        threshold_analysis.append({
            'threshold': threshold,
            'net_impact': net_impact,
            'approval_rate': (tn_t + fn_t) / len(y_test),
            'default_rate': fn_t / (tn_t + fn_t) if (tn_t + fn_t) > 0 else 0
        })
    
    threshold_df = pd.DataFrame(threshold_analysis)
    best_threshold_row = threshold_df.loc[threshold_df['net_impact'].idxmax()]
    
    print("\nThreshold Analysis:")
    print(threshold_df.round(2))
    
    print(f"\n🎯 Optimal Threshold: {best_threshold_row['threshold']:.2f}")
    print(f"   Expected Net Impact: ${best_threshold_row['net_impact']:,.0f}")
    print(f"   Approval Rate: {best_threshold_row['approval_rate']:.1%}")
    print(f"   Default Rate: {best_threshold_row['default_rate']:.1%}")

## 9. SHAP Model Interpretability

In [ ]:
# SHAP analysis for model interpretability
if 'best_model_instance' in locals():
    print("""=""*50)
    print("SHAP MODEL INTERPRETABILITY")
    print("""=""*50)
    
    try:
        # Sample data for SHAP analysis (to speed up computation)
        sample_size = min(100, len(X_test_processed))
        sample_indices = np.random.choice(len(X_test_processed), sample_size, replace=False)
        X_sample = X_test_processed.iloc[sample_indices]
        
        print(f"Running SHAP analysis on {sample_size} samples...")
        
        # Create SHAP explainer
        if best_model == 'XGBoost' or best_model == 'Random Forest':
            explainer = shap.TreeExplainer(best_model_instance)
        else:
            explainer = shap.LinearExplainer(best_model_instance, X_sample)
        
        # Calculate SHAP values
        shap_values = explainer.shap_values(X_sample)
        
        # For binary classification, get values for positive class
        if isinstance(shap_values, list):
            shap_values = shap_values[1]
        
        # Summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_sample, plot_type="bar", max_display=15)
        plt.title(f'SHAP Feature Importance - {best_model}')
        plt.tight_layout()
        plt.show()
        
        # Detailed summary plot
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_sample, max_display=15)
        plt.title(f'SHAP Summary Plot - {best_model}')
        plt.tight_layout()
        plt.show()
        
        # Individual explanations for a few samples
        print("\nIndividual Sample Explanations:")
        for i in range(min(3, len(X_sample))):
            sample_idx = sample_indices[i]
            actual_outcome = 'Charged Off' if y_test.iloc[sample_idx] == 1 else 'Fully Paid'
            predicted_prob = y_proba[sample_idx]
            
            print(f"\n--- Sample {i+1} ---")
            print(f"Actual Outcome: {actual_outcome}")
            print(f"Predicted Default Probability: {predicted_prob:.3f}")
            
            # Force plot
            plt.figure(figsize=(12, 3))
            shap.force_plot(explainer.expected_value, shap_values[i], X_sample.iloc[i], 
                          matplotlib=True, show=False)
            plt.title(f'SHAP Force Plot - Sample {i+1}')
            plt.tight_layout()
            plt.show()
    
    except Exception as e:
        print(f"SHAP analysis failed: {str(e)}")
        print("This might be due to memory limitations or model compatibility issues.")

## 10. Conclusions and Recommendations

In [ ]:
# Generate final summary and recommendations
if 'results' in locals() and 'best_model' in locals():
    print("""=""*50)
    print("FINAL CONCLUSIONS AND RECOMMENDATIONS")
    print("""=""*50)
    
    best_result = results[best_model]
    metrics = best_result['metrics']
    
    print(f"\n📈 MODEL PERFORMANCE SUMMARY")
    print(f"   Best Model: {best_model}")
    print(f"   Accuracy: {metrics['accuracy']:.1%}")
    print(f"   Precision: {metrics['precision']:.1%}")
    print(f"   Recall: {metrics['recall']:.1%}")
    print(f"   F1-Score: {metrics['f1']:.1%}")
    print(f"   ROC-AUC: {metrics['roc_auc']:.1%}")
    
    print(f"\n🎯 KEY BUSINESS INSIGHTS")
    
    # Most important risk factors (if feature importance available)
    if 'feature_importance' in locals():
        print("\nTop 5 Risk Factors:")
        for i, (_, row) in enumerate(feature_importance.head(5).iterrows()):
            print(f"   {i+1}. {row['feature']}")
    
    # Risk profiles from EDA
    print("\nHigh-Risk Borrower Profile:")
    print("   • Low FICO scores (< 680)")
    print("   • High debt-to-income ratio (> 30%)")
    print("   • Short employment history (< 2 years)")
    print("   • High credit utilization (> 80%)")
    print("   • Recent delinquencies or inquiries")
    
    print("\nLow-Risk Borrower Profile:")
    print("   • High FICO scores (> 760)")
    print("   • Low debt-to-income ratio (< 15%)")
    print("   • Stable employment (> 5 years)")
    print("   • Home ownership (mortgage or owned)")
    print("   • Clean credit history")
    
    print(f"\n💡 BUSINESS RECOMMENDATIONS")
    print("\n1. Risk-Based Pricing:")
    print("   • Implement tiered interest rates based on risk scores")
    print("   • Higher rates for high-risk applicants to compensate for increased default risk")
    
    print("\n2. Underwriting Guidelines:")
    print("   • Minimum FICO score of 680 for standard loans")
    print("   • Maximum DTI ratio of 40% for loan approval")
    print("   • Require income verification for all applicants")
    print("   • Minimum employment length of 6 months")
    
    print("\n3. Portfolio Management:")
    print("   • Diversify loan portfolio by risk grade")
    print("   • Monitor portfolio performance and adjust risk thresholds")
    print("   • Implement early warning systems for at-risk loans")
    
    print("\n4. Model Governance:")
    print("   • Regular model retraining (quarterly or when performance degrades)")
    print("   • Monitor for model drift and bias")
    print("   • Maintain model documentation and validation procedures")
    
    print("\n5. Future Enhancements:")
    print("   • Incorporate alternative data sources (employment verification, bank statements)")
    print("   • Develop real-time risk monitoring dashboard")
    print("   • Implement automated decision workflows for low-risk applications")
    
    # Save the best model
    try:
        import joblib
        model_data = {
            'model': best_model_instance,
            'scaler': scaler,
            'encoders': encoders,
            'feature_columns': X_train.columns.tolist(),
            'processed_columns': X_train_processed.columns.tolist(),
            'metrics': metrics,
            'model_type': best_model
        }
        
        joblib.dump(model_data, '../models/best_model.pkl')
        print(f"\n✅ Best model saved to '../models/best_model.pkl'")
    except Exception as e:
        print(f"\n⚠️  Could not save model: {str(e)}")
    
    print(f"\n" + "="*50)
    print("ANALYSIS COMPLETE")
    print("""=""*50)
    
    print(f"\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Analyst: Rafsamjani Anugrah")
    print(f"Project: Credit Risk Prediction for ID/X Partners")
    print(f"Status: Successfully completed with {best_model} model")
else:
    print("No model results available for final analysis.")